In [1]:
import pandas as pd
import numpy as np
from datetime import time
from numba import njit, prange
from itertools import product
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from bokeh.plotting import figure, show, output_file, curdoc
from bokeh.models import ColumnDataSource, LinearAxis
from bokeh.layouts import column

In [2]:
data_es = pd.read_parquet('data_es.parquet', engine='pyarrow')
data_nq = pd.read_parquet('data_nq.parquet', engine='pyarrow')

In [3]:
# display all columns
pd.set_option('display.max_columns', None)

In [4]:
data_es.head()

,Open,High,Low,Close,Volume,NumOfTrades,OHLCAvg,HLCAvg,HLAvg,BidVolume,AskVolume,fsvwap,dPOC,dVAH,dVAL,rthvwap,onPOC,onVAH,onVAL,dHi,dLo,dMid,IBH5,IBL5,IBH15,IBL15,IBH30,IBL30,IBH,IBL,ONH,ONL,ONMID,pHi,pLo,pMid,pCl,score,AvgTradeSize
2018-08-02 08:30:00,3095.50,3095.75,3093.25,3093.75,5183.0,2017.0,3094.56,3094.25,3094.50,3042.0,2141.0,3098.1,3093.75,3095.00,3093.50,3094.39,3093.25,3100.75,3089.5,3095.75,3093.25,3094.500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3113.25,3089.25,3101.25,NaN,NaN,NaN,NaN,0.0,2.57
2018-08-02 08:30:15,3093.50,3094.75,3093.50,3094.75,2644.0,971.0,3094.13,3094.33,3094.13,1265.0,1379.0,3098.1,3094.50,3094.75,3093.50,3094.36,3093.25,3100.75,3089.5,3095.75,3093.25,3094.500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3113.25,3089.25,3101.25,NaN,NaN,NaN,NaN,0.0,2.72
2018-08-02 08:30:30,3094.75,3095.25,3094.50,3094.50,1298.0,547.0,3094.75,3094.75,3094.88,862.0,436.0,3098.1,3094.50,3095.00,3093.75,3094.43,3093.25,3100.75,3089.5,3095.75,3093.25,3094.500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3113.25,3089.25,3101.25,NaN,NaN,NaN,NaN,25.0,2.37
2018-08-02 08:30:45,3094.25,3095.00,3093.75,3094.25,2162.0,757.0,3094.31,3094.33,3094.38,1143.0,1019.0,3098.0,3094.50,3094.75,3093.75,3094.43,3093.25,3100.75,3089.5,3095.75,3093.25,3094.500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3113.25,3089.25,3101.25,NaN,NaN,NaN,NaN,5.0,2.86
2018-08-02 08:31:00,3094.25,3096.00,3094.00,3095.75,1970.0,840.0,3095.00,3095.25,3095.00,947.0,1023.0,3098.0,3094.25,3095.00,3093.75,3094.55,3093.25,3100.75,3089.5,3096.00,3093.25,3094.625,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3113.25,3089.25,3101.25,NaN,NaN,NaN,NaN,5.0,2.35


In [5]:
data_nq.head()

,Open,High,Low,Close,Volume,NumOfTrades,OHLCAvg,HLCAvg,HLAvg,BidVolume,AskVolume,fsvwap,dPOC,dVAH,dVAL,rthvwap,onPOC,onVAH,onVAL,dHi,dLo,dMid,IBH5,IBL5,IBH15,IBL15,IBH30,IBL30,IBH,IBL,ONH,ONL,ONMID,pHi,pLo,pMid,pCl,score,AvgTradeSize
2018-08-02 08:30:00,8762.25,8763.25,8759.00,8761.50,1246.0,1100.0,8761.50,8761.25,8761.13,542.0,704.0,8776.0,8761.75,8763.00,8760.50,8761.25,8757.25,8786.75,8750.0,8763.25,8759.0,8761.125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8827.5,8748.5,8788.0,NaN,NaN,NaN,NaN,0.0,1.13
2018-08-02 08:30:15,8761.75,8767.75,8761.50,8766.75,1053.0,895.0,8764.44,8765.33,8764.63,418.0,635.0,8775.8,8761.75,8765.00,8759.00,8763.21,8757.25,8786.75,8750.0,8767.75,8759.0,8763.375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8827.5,8748.5,8788.0,NaN,NaN,NaN,NaN,5.0,1.18
2018-08-02 08:30:30,8767.00,8769.00,8764.25,8764.25,810.0,742.0,8766.13,8765.83,8766.63,387.0,423.0,8775.7,8767.00,8768.75,8762.00,8764.32,8757.25,8786.75,8750.0,8769.00,8759.0,8764.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8827.5,8748.5,8788.0,NaN,NaN,NaN,NaN,15.0,1.09
2018-08-02 08:30:45,8764.25,8765.75,8761.00,8763.00,881.0,825.0,8763.50,8763.25,8763.38,490.0,391.0,8775.6,8767.00,8767.75,8761.75,8764.14,8757.25,8786.75,8750.0,8769.00,8759.0,8764.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8827.5,8748.5,8788.0,NaN,NaN,NaN,NaN,-10.0,1.07
2018-08-02 08:31:00,8763.00,8769.25,8762.75,8766.75,947.0,866.0,8765.44,8766.25,8766.00,416.0,531.0,8775.5,8767.00,8768.75,8762.50,8764.71,8757.25,8786.75,8750.0,8769.25,8759.0,8764.125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8827.5,8748.5,8788.0,NaN,NaN,NaN,NaN,5.0,1.09


In [6]:
# plot with plotly

date = "2024-08-08"
df = data_es[data_es.index.date == pd.to_datetime(date).date()]

# Filter data to show only after 09:30:00
df_after_0930 = df[df.index.time >= pd.to_datetime('09:30:00').time()]

# Create subplots
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                    vertical_spacing=0.03, subplot_titles=('OHLC', 'Score'),
                    row_width=[0.2, 0.7])

# Plot OHLC candlestick chart
fig.add_trace(go.Candlestick(x=df.index,
                             open=df['Open'],
                             high=df['High'],
                             low=df['Low'],
                             close=df['Close'],
                             name='OHLC'),
              row=1, col=1)

# Add lines (IBH and IBL only after 09:30:00)
fig.add_trace(go.Scatter(x=df_after_0930.index, y=df_after_0930['IBH'], mode='lines', name='IBH', line=dict(color='orange', width=1)), row=1, col=1)
fig.add_trace(go.Scatter(x=df_after_0930.index, y=df_after_0930['IBL'], mode='lines', name='IBL', line=dict(color='orange', width=1)), row=1, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df['fsvwap'], mode='lines', name='fsvwap', line=dict(color='yellow', width=1)), row=1, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df['rthvwap'], mode='lines', name='rthvwap', line=dict(color='yellow', width=1)), row=1, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df['dPOC'], mode='lines', name='dPOC', line=dict(color='magenta', width=1)), row=1, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df['dVAH'], mode='lines', name='dVAH', line=dict(color='lightblue', width=1)), row=1, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df['dVAL'], mode='lines', name='dVAL', line=dict(color='lightblue', width=1)), row=1, col=1)


# Plot score as a histogram
fig.add_trace(go.Bar(x=df.index, y=df['score'], name='Score', marker_color='darkred'),
              row=2, col=1)

# Layout adjustments (Grid lines, price scale on the right, and time intervals)
fig.update_layout(
    xaxis=dict(
        dtick=1800 * 1000  # Grid lines every 30 minutes (1800 seconds)
    ),
    yaxis=dict(
        dtick=5,  # Grid lines every 5 points
        side='right'  # Price scale on the right
    ),
    title=f'{date} OHLC and Trend Score',
    yaxis_title='Price',
    #xaxis_title='Time',
    xaxis_rangeslider_visible=False,
    template='plotly_dark',
    height=1080,
    width=1920
)

fig.update_xaxes(title_text='Time', row=2, col=1, title_standoff=5) 

# Adjust y-axis for the score plot
fig.update_yaxes(title_text='Score', side='right', row=2, col=1)

# Show the plot
fig.show()


In [10]:
# plot with bokeh

# Prepare data for the given date range
start_date = "2024-08-07"
end_date = "2024-08-08"
df = data_es[(data_es.index.date >= pd.to_datetime(start_date).date()) & 
             (data_es.index.date <= pd.to_datetime(end_date).date())]

# Create the main OHLC chart with 4K resolution dimensions
p1 = figure(x_axis_type="datetime", title="OHLC", height=800, width=1920)

# Score plot
p2 = figure(x_axis_type="datetime", title="Score", height=230, width=1920, x_range=p1.x_range)

# Process each day separately within the given date range
for date, group in df.groupby(df.index.date):
    source = ColumnDataSource(group)
    
    # Candlestick chart
    inc = group['Close'] > group['Open']
    dec = group['Open'] > group['Close']
    
    # Dynamically calculate the width based on time intervals between points
    w = (group.index[1] - group.index[0]).total_seconds() * 1000 * 0.8  # 80% of time interval as width
    
    # Segments for High and Low
    p1.segment(x0='index', y0='High', x1='index', y1='Low', color="black", source=source)
    
    # Candlesticks for price movement
    p1.vbar(x='index', width=w, top='Open', bottom='Close', fill_color="lightblue", line_color="black", source=ColumnDataSource(group[inc]))
    p1.vbar(x='index', width=w, top='Close', bottom='Open', fill_color="red", line_color="black", source=ColumnDataSource(group[dec]))
    
    # Plot IBH and IBL lines after 09:30
    df_after_0930 = group[group.index.time >= pd.to_datetime('09:30:00').time()]
    source_after_0930 = ColumnDataSource(df_after_0930)
    p1.line('index', 'IBH', source=source_after_0930, line_width=2, color='orange', legend_label='IBH')
    p1.line('index', 'IBL', source=source_after_0930, line_width=2, color='orange', legend_label='IBL')
    
    # Plot additional lines
    p1.line('index', 'fsvwap', source=source, line_width=2, color='yellow', legend_label='fsvwap')
    p1.line('index', 'rthvwap', source=source, line_width=2, color='yellow', legend_label='rthvwap')
    p1.line('index', 'dPOC', source=source, line_width=2, color='magenta', legend_label='dPOC')
    p1.line('index', 'dVAH', source=source, line_width=2, color='lightblue', legend_label='dVAH')
    p1.line('index', 'dVAL', source=source, line_width=2, color='lightblue', legend_label='dVAL')
    # onPOC
    p1.line('index', 'onPOC', source=source, line_width=2, color='green', legend_label='onPOC')
    # ONH and ONL
    # p1.line('index', 'ONH', source=source, line_width=2, color='green', legend_label='ONH')
    # p1.line('index', 'ONMID', source=source, line_width=2, color='green', legend_label='ONMID')
    # p1.line('index', 'ONL', source=source, line_width=2, color='green', legend_label='ONL')
    # onVAL and onVAH
    # p1.line('index', 'onVAL', source=source, line_width=2, color='green', legend_label='onVAL')
    # p1.line('index', 'onVAH', source=source, line_width=2, color='green', legend_label='onVAH')
    
    # Plot score as a histogram
    p2.vbar(x='index', top='score', width=w, color="darkred", source=source)

# Add a secondary y-axis for the price scale on the right side
p1.extra_y_ranges = {"price_range": p1.y_range}
p1.add_layout(LinearAxis(y_range_name="price_range"), 'right')

# Add a secondary y-axis for the score scale on the right side
p2.extra_y_ranges = {"score_range": p2.y_range}
p2.add_layout(LinearAxis(y_range_name="score_range"), 'right')

# Customize the legend (position it on the right and reduce font size)
p1.legend.location = "top_left"
p1.legend.orientation = "vertical"
p1.legend.label_text_font_size = "6pt"
p1.add_layout(p1.legend[0], 'right')

# Apply dark theme to the entire plot
curdoc().theme = 'dark_minimal'

# Combine the OHLC chart and score chart vertically
layout = column(p1, p2)

# Output the plot to an HTML file and display it in the web browser
output_file("bokeh_plot.html")
show(layout)


In [8]:
# Filter data from 2024-01-01 to 2024-08-08
start_date = "2024-01-01"
end_date = "2024-08-08"
df_es_filtered = data_es[(data_es.index.date >= pd.to_datetime(start_date).date()) & 
                         (data_es.index.date <= pd.to_datetime(end_date).date())]
df_nq_filtered = data_nq[(data_nq.index.date >= pd.to_datetime(start_date).date()) & 
                         (data_nq.index.date <= pd.to_datetime(end_date).date())]

# Calculate correlation for 'Close' columns
correlation_close = df_es_filtered['Close'].corr(df_nq_filtered['Close'])
print(f"Correlation (ES Close vs NQ Close): {correlation_close:.2f}")

# Calculate correlation for 'score' columns
correlation_score = df_es_filtered['score'].corr(df_nq_filtered['score'])
print(f"Correlation (ES Score vs NQ Score): {correlation_score:.2f}")

Correlation (ES Close vs NQ Close): 0.95
Correlation (ES Score vs NQ Score): 0.86


NEXT STEP: ADVANCED
CHECK: https://github.com/pythonicflow/SierraChartScidToPostgres


In [9]:
# import asyncio
# import asyncpg
# import dash
# from dash import dcc, html
# from dash.dependencies import Input, Output
# import pandas as pd
# from datetime import datetime, timedelta
# import pytz
# import plotly.graph_objects as go
# from collections import defaultdict
# from plotly.subplots import make_subplots

# # Sierra Chart epoch start time
# sc_epoch = datetime(1899, 12, 30, tzinfo=pytz.UTC)

# # Function to convert Sierra Chart datetime to human-readable datetime in Central Time
# def convert_sc_datetime(sc_datetime_ms):
#     delta = timedelta(microseconds=sc_datetime_ms)
#     actual_datetime = sc_epoch + delta
#     actual_datetime = actual_datetime.replace(tzinfo=pytz.UTC)  # Set timezone to UTC
#     local_tz = pytz.timezone('America/Chicago')  # Convert to Central Time
#     local_datetime = actual_datetime.astimezone(local_tz)
#     return local_datetime

# # Function to create a database connection
# async def create_db_connection():
#     return await asyncpg.connect(
#         host=config.DB_HOST,
#         port=config.DB_PORT,
#         user=config.DB_USER,
#         password=config.DB_PASSWORD,
#         database=config.DB_NAME
#     )

# # Function to calculate start and end SC times for the given date
# def get_sc_time(date):
#     date = datetime.strptime(date, "%Y-%m-%d")
#     local_tz = pytz.timezone('America/Chicago')
#     start_of_day = local_tz.localize(date.replace(hour=8, minute=30, second=0, microsecond=0))
#     end_of_day = local_tz.localize(date.replace(hour=15, minute=15, second=0, microsecond=0))
#     start_delta = start_of_day.astimezone(pytz.UTC) - sc_epoch
#     end_delta = end_of_day.astimezone(pytz.UTC) - sc_epoch
#     sc_start_of_day = start_delta.total_seconds() * 1e6  # Convert to microseconds
#     sc_end_of_day = end_delta.total_seconds() * 1e6  # Convert to microseconds
#     return int(sc_start_of_day), int(sc_end_of_day)

# # Function to fetch data from the database
# async def fetch_data(date, min_quantity):
#     start_time, end_time = get_sc_time(date)
#     conn = await create_db_connection()
#     rows = await conn.fetch(f'SELECT * FROM public."esu24dot" WHERE "scdatetime" BETWEEN {start_time} AND {end_time} AND "quantity" >= {min_quantity} ORDER BY "scdatetime" DESC;')
#     df = pd.DataFrame(rows, columns=['scdatetime', 'price', 'quantity', 'side'])
#     df['scdatetime'] = df['scdatetime'].apply(convert_sc_datetime)
#     df['scdatetime_str'] = df['scdatetime'].dt.strftime('%H:%M:%S.%f')
#     df = df.sort_values('scdatetime')
#     await conn.close()
#     return df

# app = dash.Dash(__name__)

# app.layout = html.Div([
#     html.Label('Date: '),
#     dcc.Input(id='date-input', type='text', value='2024-07-08'),  # Initial date value
#     html.Label(' Minimum Quantity: '),
#     dcc.Input(id='min-quantity-input', type='number', value=10),  # Initial minimum quantity value
#     dcc.Graph(id='live-graph')
# ])

# # Function to calculate Point of Control (POC)
# def calculate_poc(df):
#     price_volumes = defaultdict(int)
#     poc_changes = []
#     times = []

#     for index, row in df.iterrows():
#         price = row['price']
#         quantity = row['quantity']
#         current_time = row['scdatetime']

#         price_volumes[price] += quantity

#         # Calculate POC for the current data
#         poc_price = max(price_volumes, key=price_volumes.get)
#         poc_changes.append(poc_price)
#         times.append(current_time)

#     return times, poc_changes

# # Callback function to update the graph based on user input
# @app.callback(Output('live-graph', 'figure'),
#               [Input('date-input', 'value'), Input('min-quantity-input', 'value')])
# def update_graph_live(date, min_quantity):
#     df = asyncio.run(fetch_data(date, min_quantity))
#     color_map = {0: 'lightcoral', 1: 'lightblue'}

#     df = df.set_index('scdatetime')
#     df = df.between_time('08:30', '15:15')
#     df = df.reset_index()

#     fig = make_subplots(
#         rows=1, cols=2,
#         column_widths=[0.9, 0.1],
#         specs=[[{"type": "scatter"}, {"type": "bar"}]],
#         horizontal_spacing=0.015,
#         shared_yaxes=True  # Align y-axes
#     )

#     fig.add_trace(go.Scatter(
#         x=df['scdatetime'],
#         y=df['price'],
#         mode='markers',
#         marker=dict(
#             size=df['quantity'],
#             color=df['side'].apply(lambda x: color_map[x])
#         ),
#         hovertemplate=
#         '<i>Price</i>: %{y}'+
#         '<br><b>Quantity</b>: %{marker.size}<br>',
#         name='Trades'
#     ), row=1, col=1)

#     # Draw the POC line
#     times, poc_changes = calculate_poc(df)
#     fig.add_trace(go.Scatter(x=times, y=poc_changes, mode='lines', name='POC', line=dict(color='yellow', width=5)), row=1, col=1)

#     # Add Volume Profile
#     df_bid = df[df['side'] == 0].groupby('price')['quantity'].sum().reset_index()
#     df_ask = df[df['side'] == 1].groupby('price')['quantity'].sum().reset_index()

#     fig.add_trace(go.Bar(x=df_bid['quantity'], y=df_bid['price'], orientation='h', 
#                          marker=dict(color='lightcoral'), name='Bid'), row=1, col=2)
#     fig.add_trace(go.Bar(x=df_ask['quantity'], y=df_ask['price'], orientation='h', 
#                          marker=dict(color='lightblue'), name='Ask'), row=1, col=2)

#     # Calculate the min and max prices for the day and adjust y-axis range
#     min_price = df['price'].min() - 5  # 5 units below the minimum price
#     max_price = df['price'].max() + 5  # 5 units above the maximum price
#     dtick_value = 1.00  # Set tick interval for y-axis

#     fig.update_layout(
#         template="ggplot2",
#         plot_bgcolor='#333333',
#         paper_bgcolor='#333333',
#         font=dict(family="Courier New, monospace", size=18, color="white"),
#         title_font=dict(size=24, family='Verdana, sans-serif', color='lightblue'),
#         legend_title_font=dict(family="Times New Roman, Times, serif", size=16, color='lightgreen'),
#         xaxis=dict(showgrid=True, gridwidth=1, gridcolor='rgba(211, 211, 211, 0.3)'),
#         yaxis=dict(showgrid=True, gridwidth=1, gridcolor='rgba(211, 211, 211, 0.3)', tickmode='linear', tick0=0, dtick=dtick_value, side='right', range=[min_price, max_price]),
#         hovermode='closest',
#         autosize=False,
#         width=5730,
#         height=3080,
#         barmode='stack',
#         bargap=0.0,
#         bargroupgap=0.0
#     )

#     # Set the x-axis range to cover the market hours
#     start_range = datetime.strptime(date, "%Y-%m-%d").replace(hour=8, minute=30).isoformat()
#     end_range = datetime.strptime(date, "%Y-%m-%d").replace(hour=15, minute=15).isoformat()
#     fig.update_xaxes(range=[start_range, end_range])

#     return fig

# if __name__ == '__main__':
#     app.run_server(debug=True, port=8052)


# OUTPUT : image.png
